In [1]:
import pandas as pd
from optbinning import OptimalBinning
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Mar 01 10:58:09 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 01 10:58:09 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


### Load data and repeat the preprocess.

In [2]:
file_path = "/hdd1/sams/diabetes/diabetes.arff"
with open(file_path, 'r') as f:
    lines = f.readlines()

In [3]:
flag = False
rows = []
for line in lines:
    if line.strip() == "@data":
        flag = True
        continue
    
    if not flag:
        continue
    
    row = [i for i in line.strip().split(',')]
    rows.append(row) 

In [4]:
data = np.array(rows)
df = pd.DataFrame(data)

In [5]:
y_map = {"'NO'":0, "'<30'":1, "'>30'":1}
df[49] = df[49].map(y_map)

In [6]:
patient_col = df[1].values
d = {}
for i in patient_col:
    if i not in d:
        d[i] = 0
    d[i] += 1

In [7]:
map_d = {}
for k,v in d.items():
    if v > 1:
        map_d[k] = 1
    else:
        map_d[k] = 0

In [8]:
# df[1] = df[1].map(d)
df[1] = df[1].map(map_d)

In [9]:
# diag_1 18
print(len(df[18].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[18].values,y)
print(optb.status)

x = optb.transform(df[18], metric="indices")
print(max(x))
df[18] = x

717
OPTIMAL
88


In [10]:
# diag_2 19
print(len(df[19].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[19].values,y)
print(optb.status)
x = optb.transform(df[19], metric="indices")
print(max(x))
df[19] = x

749
OPTIMAL
80


In [11]:
# diag_3 20
print(len(df[20].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[20].values,y)
print(optb.status)
x = optb.transform(df[20], metric="indices")
print(max(x))
df[20] = x

790
OPTIMAL
84


In [12]:
del df[0]
Y = df[49]
del df[49]
X = df

In [13]:
col_to_value2token = {}
cnt = 0
for col in X.columns:
    col_values = X[col].values
    col_to_value2token[col] = {}
    col_values_set = set(col_values)
    for val in col_values_set:
        if val not in col_to_value2token[col]:
            col_to_value2token[col][val] = cnt
            cnt += 1

In [15]:
# sql 14
predicate_bitmap = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
sql = [833, 834, 835, 836, 837, 34, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 825, 878, 830, 880]

In [16]:
def parse_sql(predicate_bitmap, sql):
    for col_idx, i in enumerate(predicate_bitmap):
        if i == 0:
            continue
        cname = "V"+ str(col_idx + 1)
        value2token = col_to_value2token[X.columns[col_idx]]
        print(f"value to token:{value2token}")
        print(f'Column {cname}, token {sql[col_idx]}')

In [17]:
parse_sql(predicate_bitmap, sql)

value to token:{'1': 31, '6': 32, '8': 33, '4': 34, '3': 35, '7': 36, '5': 37, '2': 38}
Column V6, token 34
value to token:{"'Steady'": 826, "'No'": 827}
Column V45, token 825
value to token:{"'Ch'": 830, "'No'": 831}
Column V47, token 830


In [18]:
X[X.columns[5]].value_counts()

6
1    53990
3    18869
2    18480
6     5291
5     4785
8      320
7       21
4       10
Name: count, dtype: int64

In [19]:
X[X.columns[44]].value_counts()

45
'No'        101764
'Steady'         2
Name: count, dtype: int64

In [21]:
X[X.columns[46]].value_counts()

47
'No'    54755
'Ch'    47011
Name: count, dtype: int64

In [ ]:
# change = 'No' && admission_type = 3 && metformin.rosiglitazone = 'Ch'

In [23]:
# 16
predicate_bitmap = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sql = [833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 531, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880]
parse_sql(predicate_bitmap, sql)

value to token:{0: 480, 1: 481, 2: 482, 3: 483, 4: 484, 5: 485, 6: 486, 7: 487, 8: 488, 9: 489, 10: 490, 11: 491, 12: 492, 13: 493, 14: 494, 15: 495, 16: 496, 17: 497, 18: 498, 19: 499, 20: 500, 21: 501, 22: 502, 23: 503, 24: 504, 25: 505, 26: 506, 27: 507, 28: 508, 29: 509, 30: 510, 31: 511, 32: 512, 33: 513, 34: 514, 35: 515, 36: 516, 37: 517, 38: 518, 39: 519, 40: 520, 41: 521, 42: 522, 43: 523, 44: 524, 45: 525, 46: 526, 47: 527, 48: 528, 49: 529, 50: 530, 51: 531, 52: 532, 53: 533, 54: 534, 55: 535, 56: 536, 57: 537, 58: 538, 59: 539, 60: 540, 61: 541, 62: 542, 63: 543, 64: 544, 65: 545, 66: 546, 67: 547, 68: 548, 69: 549, 70: 550, 71: 551, 72: 552, 73: 553, 74: 554, 75: 555, 76: 556, 77: 557, 78: 558, 79: 559, 80: 560, 81: 561, 82: 562, 83: 563, 84: 564, 85: 565, 86: 566, 87: 567, 88: 568}
Column V18, token 531


In [59]:
X[X.columns[17]].value_counts().to_dict()[50]


615

In [ ]:
# diag_1 = '50'

In [64]:
# 17
predicate_bitmap = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
sql = [833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 458, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 827, 879, 880]
parse_sql(predicate_bitmap, sql)

value to token:{'1': 426, '6': 427, '8': 428, '3': 429, '10': 430, '12': 431, '42': 432, '28': 433, '18': 434, '76': 435, '15': 436, '19': 437, '25': 438, '54': 439, '4': 440, '22': 441, '7': 442, '64': 443, '5': 444, '13': 445, '37': 446, '16': 447, '2': 448, '0': 449, '63': 450, '46': 451, '24': 452, '29': 453, '9': 454, '21': 455, '20': 456, '11': 457, '14': 458}
Column V16, token 458
value to token:{"'Steady'": 828, "'No'": 829}
Column V46, token 827


In [70]:
X[X.columns[15]].value_counts().to_dict()['20']

4

In [68]:
X[X.columns[45]].value_counts()

46
'No'        101765
'Steady'         1
Name: count, dtype: int64

In [ ]:
# metformin.pioglitazone = 'No' && number_outpatients = 